In [1]:
import plotly.express as px
import pandas as pd

from dataframes_new import get_df
from ftplib import FTP
from io import BytesIO
from os import getenv
from config import host, port, user, pwd

In [2]:
host = getenv("FTPHOST")
port = getenv("FTPPORT")
user = getenv("FTPUSER")
pwd = getenv("FTPPWD")

In [5]:
dfname = "ac_util"

with FTP() as ftp:
    ftp.set_pasv(False)
    ftp.connect(host, int(port))
    ftp.login(user, pwd)
    ftp.cwd("amos_aims_intfc/emr_dash_test")
    file = BytesIO()
    # Store the retrieved file as binary in memory
    ftp.retrbinary(f"RETR {dfname}.csv", file.write)
    # Get the very first stream position
    file.seek(0)
    df0 = pd.read_csv(file)
    ftp.quit()


EOFError: 

In [17]:
raw = df0
raw['Start'] = pd.to_datetime(raw['Start'])
df = raw[(raw['Start'].dt.month > datetime.now().month-3) & (raw['Start'].dt.year == datetime.now().year)].drop(columns=['A/C Group', 'End'])
df

,Start,A/C,AC-Type,TAH/Start,TAH/End,TAH/Diff,TAC/Start,TAC/End,TAC/Diff,Oper.Days,Av.Hours/Day,Av.Cycles/Day,Av.Length/Cycle,Rev.Hours,Rev.Cycles
0,2023-07-01,PWC,32S,28926.36667,29083.88333,157.516667,18782,18900,118,24,6.566667,4.916667,1.333333,157.516667,118
1,2023-07-01,PWD,32S,30057.08333,30208.65000,151.566667,19829,19949,120,23,6.583333,5.217391,1.266667,151.566667,120
2,2023-07-01,PWA,32S,39960.28333,40103.76667,143.483333,20929,21037,108,24,5.983333,4.500000,1.333333,143.483333,108
3,2023-07-01,PWG,32S,39294.95000,39440.98333,146.033333,20018,20132,114,24,6.083333,4.750000,1.283333,146.033333,114
4,2023-07-01,PWH,32S,47680.06667,47836.76667,156.700000,17222,17344,122,24,6.533333,5.083333,1.283333,156.700000,122
5,2023-07-01,PWE,32S,46686.03333,46828.83333,142.800000,15141,15247,106,24,5.950000,4.416667,1.350000,142.800000,106
6,2023-07-01,PWF,32S,28194.61667,28214.90000,20.283333,17789,17798,9,4,5.066667,2.250000,2.250000,7.866667,6


In [33]:
period = 4

# Get the dataframe and filter the last 'period' months
df = get_df("ac_util")
df["Start"] = pd.to_datetime(df["Start"])
df = df[
    (df["Start"].dt.month > datetime.now().month - period)
    & (df["Start"].dt.year == datetime.now().year)
].drop(columns=["A/C Group", "End"]).sort_values("A/C")
annotTAH = df["TAH/Diff"].round(2)
fig = px.bar(
    df, x="Start", y="TAH/Diff", color="A/C", barmode="group", text=annotTAH,
)

fig.show()